In [3]:
import tensorflow as tf
import time

In [2]:
tf.__version__

'2.14.0'

In [14]:
class FileDataset(tf.data.Dataset):
    def read_file_in_batches(num_samples):
        # Opening the file
        time.sleep(0.03)
        for sample_idx in range(num_samples):
            # Reading data (line, record) from the file
            time.sleep(0.015)
            yield (sample_idx,)
    def __new__(cls, num_samples=3): #always called
        print("i always run")
        return tf.data.Dataset.from_generator(
             cls.read_file_in_batches,
            output_signature = tf.TensorSpec(shape = (1,), dtype = tf.int64),
            args=(num_samples,)
        )

In [15]:
obj = FileDataset()

i always run


In [16]:
def benchmark(dataset, num_epochs=2):
    for epoch_num in range(num_epochs): ##training
        for sample in dataset:
            # Performing a training step
            time.sleep(0.01)

In [18]:
%%timeit ##without prefetch
benchmark(FileDataset())

i always run
i always run
i always run
i always run
i always run
i always run
i always run
i always run
247 ms ± 11.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%%timeit
benchmark(FileDataset().prefetch(1))

i always run
i always run
i always run
i always run
i always run
i always run
i always run
i always run
237 ms ± 1.63 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%%timeit
benchmark(FileDataset().prefetch(tf.data.AUTOTUNE))

i always run
i always run
i always run
i always run
i always run
i always run
i always run
i always run
258 ms ± 12.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Cache

In [23]:
dataset = tf.data.Dataset.range(5)
dataset = dataset.map(lambda x: x**2)
dataset = dataset.cache("mycache.txt")
# The first time reading through the data will generate the data using
# `range` and `map`.
list(dataset.as_numpy_iterator()) #reading it through cache, no map

[0, 1, 4, 9, 16]

In [25]:
def mapped_function(s):
    # Do some hard pre-processing
    tf.py_function(lambda: time.sleep(0.03), [], ()) #for tf function
    return s

In [28]:
%%timeit -r1 -n1 #r1, n1 = 1 loop
benchmark(FileDataset().map(mapped_function), 5)

i always run
1.05 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [27]:
%%timeit -r1 -n1
benchmark(FileDataset().map(mapped_function).cache(), 5)

i always run
390 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
